In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgboost
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.optimize import minimize
import matplotlib.pyplot as plt

C:\ANACONDA3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import keras_tuner as kt
import tensorflow as tf

In [3]:
#1. Data Loading
# Load the datasets
train_data = pd.read_csv('aapl_5m_train.csv')
validation_data = pd.read_csv('aapl_5m_validation.csv')

In [4]:
#2. Feature Engineering
# Calculate moving averages
train_data['MA5'] = train_data['Close'].rolling(window=5).mean()
train_data['MA10'] = train_data['Close'].rolling(window=10).mean()

# Define target variable: 1 if the next close is higher than the current close, else 0
train_data['Target'] = (train_data['Close'].shift(-1) > train_data['Close']).astype(int)

# Drop NA values
train_data.dropna(inplace=True)

# Define independent variables and target variable
X = train_data[['MA5', 'MA10']]
Y = train_data['Target']

In [5]:
#3. Modeling
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X, Y)
y_hat = log_reg.predict(X)

# SVC
svc = SVC()
svc.fit(X, Y)

# XGBoost
boosted_model = xgboost.XGBClassifier()
boosted_model.fit(X, Y)
y_hat_boost = boosted_model.predict(X)

print(confusion_matrix(Y, y_hat_boost))
print(confusion_matrix(Y, y_hat))
print(accuracy_score(Y, y_hat_boost))


# Adaptación de datos para TensorFlow
X_tf = np.array(X)
Y_tf = np.array(Y)

# Crear modelo MLP
model_mlp = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_tf.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar modelo
model_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar modelo
model_mlp.fit(X_tf, Y_tf, epochs=10, batch_size=10)


[[6084 3765]
 [4031 5643]]
[[9849    0]
 [9674    0]]
0.6006761255954515
Epoch 1/10
1953/1953 [==============================] - 5s 2ms/step - loss: 0.8022 - accuracy: 0.4977
Epoch 2/10
1953/1953 [==============================] - 4s 2ms/step - loss: 0.7384 - accuracy: 0.5043
Epoch 3/10
1953/1953 [==============================] - 3s 2ms/step - loss: 0.7204 - accuracy: 0.4981
Epoch 4/10
1953/1953 [==============================] - 4s 2ms/step - loss: 0.7034 - accuracy: 0.4981
Epoch 5/10
1953/1953 [==============================] - 3s 2ms/step - loss: 0.6961 - accuracy: 0.4999
Epoch 6/10
1953/1953 [==============================] - 3s 2ms/step - loss: 0.6932 - accuracy: 0.5042
Epoch 7/10
1953/1953 [==============================] - 3s 2ms/step - loss: 0.6931 - accuracy: 0.5030
Epoch 8/10
1953/1953 [==============================] - 3s 2ms/step - loss: 0.6932 - accuracy: 0.5045
Epoch 9/10
1953/1953 [==============================] - 3s 2ms/step - loss: 0.6932 - accuracy: 0.5045
Epoch 10/

In [7]:
#4. Optimization
def optimize_params(x: np.array) -> float:
    gamma, reg_alpha = x  # Unpack parameters
    n_estimators = 1
    model_ = xgboost.XGBClassifier(n_estimators=n_estimators,
                                   gamma=gamma,
                                   reg_alpha=reg_alpha,
                                   reg_lambda=reg_alpha)
    model_.fit(X, Y)
    y_pred = model_.predict(X)
    acc = accuracy_score(Y, y_pred)
    return -acc

bnds = ((0, 10), (1e-4, 10))
x0 = [0, 1e-3]
res = minimize(optimize_params, bounds=bnds, x0=x0, method="Nelder-Mead", tol=1e-10)
print(res)
print(res.x)

opt_model = xgboost.XGBClassifier(n_estimators=10,
                                  gamma=res.x[0],
                                  reg_alpha=res.x[1],
                                  reg_lambda=res.x[1])
opt_model.fit(X, Y)
print(confusion_matrix(Y, opt_model.predict(X)))

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                                    activation='relu', input_shape=(X_tf.shape[1],)))
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                                    activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of combinations de hiperparameters to test
    executions_per_trial=3,  # Times to run each combination
    directory='my_dir',  # Directory to save logs
    project_name='intro_to_kt')

tuner.search(X_tf, Y_tf, epochs=10, validation_split=0.2)


 final_simplex: (array([[0.00000000e+00, 1.00000000e-03],
       [5.96046448e-11, 1.00000000e-03],
       [0.00000000e+00, 1.00000001e-03]]), array([-0.51805563, -0.51805563, -0.51805563]))
           fun: -0.5180556266967167
       message: 'Optimization terminated successfully.'
          nfev: 91
           nit: 23
        status: 0
       success: True
             x: array([0.   , 0.001])
[0.    0.001]
[[5252 4597]
 [4254 5420]]


FailedPreconditionError: my_dir\intro_to_kt is not a directory

In [ ]:
#5. Combination of Models
def combine_predictions_dl(preds):
    combined = sum(preds)
    return 1 if combined >= len(preds) / 2 else 0

# Predicciones de modelos de Deep Learning
y_hat_mlp = (model_mlp.predict(X_tf) > 0.5).astype(int).flatten()
# Predicciones de la Regresión Logística
y_hat_lr = y_hat  # Ya tienes esta variable definida

# Predicciones del SVC
y_hat_svc = svc.predict(X)

# Predicciones de XGBoost
y_hat_xgb = y_hat_boost  # Ya tienes esta variable definida

# Predicciones del Modelo MLP
y_hat_mlp = (model_mlp.predict(X_tf) > 0.5).astype(int).flatten()

# Combinar predicciones de todos los modelos
combined_predictions_dl = [combine_predictions_dl([y_hat_lr[i], y_hat_svc[i], y_hat_xgb[i], y_hat_mlp[i]]) for i in range(len(y_hat_lr))]


# Combinar predicciones de todos los modelos
combined_predictions_dl = [combine_predictions_dl([y_hat_lr[i], y_hat_svc[i], y_hat_xgb[i], y_hat_mlp[i]]) for i in range(len(y_hat_lr))]


In [ ]:
#6. Backtesting
initial_cash = 1000000  # Starting cash
cash = initial_cash
stock = 0
portfolio_values = []
stop_loss_percentage = 0.05  # 5% stop loss
take_profit_percentage = 0.10  # 10% take profit
commission_rate = 0.001  # 0.1% commission
combined_predictions = combined_predictions_dl

buy_price = 0  # Precio de compra para calcular stop-loss y take-profit

for i in range(len(train_data) - 1):
    current_price = train_data['Close'].iloc[i]
    if stock > 0:
        # Calcula el cambio porcentual desde el precio de compra
        change_percentage = (current_price - buy_price) / buy_price

        # Vender si se alcanza el stop-loss o el take-profit
        if change_percentage <= -stop_loss_percentage or change_percentage >= take_profit_percentage:
            cash += stock * current_price * (1 - commission_rate)
            stock = 0
            continue

    # Señal de compra
    if combined_predictions[i] == 1 and cash >= current_price:
        stock += 1
        cash -= current_price * (1 + commission_rate)
        buy_price = current_price
    # Señal de venta
    elif combined_predictions[i] == 0 and stock > 0:
        cash += stock * current_price * (1 - commission_rate)
        stock = 0

    portfolio_values.append(cash + stock * current_price)


In [ ]:
#7. Strategy Selection & Validation
# Feature engineering for validation data
validation_data['MA5'] = validation_data['Close'].rolling(window=5).mean()
validation_data['MA10'] = validation_data['Close'].rolling(window=10).mean()
validation_data.dropna(inplace=True)
X_val = validation_data[['MA5', 'MA10']]

# Predictions on validation data
y_hat_val_lr = log_reg.predict(X_val)
y_hat_val_svc = svc.predict(X_val)
y_hat_val_xgb = opt_model.predict(X_val)

# Define la función para combinar predicciones si aún no está definida
def combine_predictions_dl(preds):
    combined = sum(preds)
    return 1 if combined >= len(preds) / 2 else 0

# Combine predictions
combined_predictions_val = [combine_predictions_dl([y_hat_val_lr[i], y_hat_val_svc[i], y_hat_val_xgb[i]]) for i in range(len(y_hat_val_lr))]

# Resto del código de backtesting en los datos de validación
cash = initial_cash
stock = 0
portfolio_values_val = []

for i in range(len(validation_data) - 1):
    current_price = validation_data['Close'].iloc[i]
    if stock > 0:
        change_percentage = (current_price - buy_price) / buy_price
        if change_percentage <= -stop_loss_percentage or change_percentage >= take_profit_percentage:
            cash += stock * current_price * (1 - commission_rate)
            stock = 0
            continue

    if combined_predictions_val[i] == 1 and cash >= current_price:
        stock += 1
        cash -= current_price * (1 + commission_rate)
        buy_price = current_price
    elif combined_predictions_val[i] == 0 and stock > 0:
        cash += stock * current_price * (1 - commission_rate)
        stock = 0

    portfolio_values_val.append(cash + stock * current_price)

portfolio_values_val.append(cash + stock * validation_data['Close'].iloc[-1])

In [ ]:
#8. Results & Conclusions
# Plot portfolio values for training data
plt.figure(figsize=(14, 7))
plt.plot(portfolio_values, label="Training Data Portfolio Value")
plt.plot(portfolio_values_val, label="Validation Data Portfolio Value", color='orange')
plt.title("Portfolio Value Over Time")
plt.xlabel("Time")
plt.ylabel("Portfolio Value")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Asegúrate de que las longitudes de las listas son correctas
print("Longitud de los datos de entrenamiento:", len(train_data))
print("Longitud de portfolio_values:", len(portfolio_values))
print("Longitud de los datos de validación:", len(validation_data))
print("Longitud de portfolio_values_val:", len(portfolio_values_val))
